In [1]:
import pandas as pd
import psycopg2

In [9]:

import settings
import local_settings
connection = psycopg2.connect(
    dbname=local_settings.DBNAME_SKILLFACTORY,
    user=local_settings.USER,
    host=local_settings.HOST,
    password=local_settings.PASSWORD,
    port=local_settings.PORT
)

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [39]:
def run_query(text_query):
    df = pd.read_sql(text_query, connection)
    display(df)

Текущее время по гринвичу

In [43]:
run_query('SELECT NOW()')

,now
0,2023-01-19 08:57:48.983152+00:00


Текущая дата по гринвичу

In [42]:
run_query('select CURRENT_DATE')
run_query('select now()::date')

,current_date
0,2023-01-19


,now
0,2023-01-19


Часовой пояс

In [31]:
run_query('show timezone')

,TimeZone
0,UCT


Текущее время в нужном часовом поясе

In [44]:
run_query("select now() at time zone 'Europe/Moscow'")

,timezone
0,2023-01-19 11:57:48.983152


In [45]:
run_query("""
with x as 
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)
select (ts at time zone 'Europe/Moscow')::date as dt_msk, ts::date as dt_utc from x
""")


,dt_msk,dt_utc
0,2019-01-01,2018-12-31


Интервалы

In [52]:
run_query("SELECT CURRENT_DATE + INTERVAL '1 DAY' as tomorrow")

,tomorrow
0,2023-01-20


Части даты/времени

In [55]:
# количество дней в интервале
run_query("SELECT INTERVAL '40 days 1 minute', EXTRACT(DAY FROM INTERVAL '40 days 1 minute')")
# неделя в году
run_query("SELECT TIMESTAMP '2001-02-16 20:38:40', EXTRACT(WEEK FROM TIMESTAMP '2001-02-16 20:38:40')")
# день недели с воскресенья
run_query("SELECT TIMESTAMP '2001-02-16 20:38:40', EXTRACT(DOW FROM TIMESTAMP '2001-02-16 20:38:40')")

,interval,date_part
0,40 days 00:01:00,40.0


,timestamp,date_part
0,2001-02-16 20:38:40,7.0


,timestamp,date_part
0,2001-02-16 20:38:40,5.0


Форматирование дат

In [59]:
run_query("SELECT to_char(current_timestamp, 'HH12:MI:SS') as time")
run_query("SELECT to_char(interval '15h 2m 12s', 'HH24:MI:SS') as time")

,time
0,09:15:11


,time
0,15:02:12


|Код форматирования	|Описание|
| :------------------- | :------------------- |
|HH24	| час (00-23)|
|MI	|минута (00-59)|
|SS	|секунда (00-59)|
|MS	|миллисекунда (000-999)|
|SSSS	|число секунд с начала суток (0-86399)|
|YYYY	|год (4 или более цифр)|
|YY	|последние две цифры года|
|month	|полное название месяца в нижнем регистре (дополненное пробелами до девяти символов)|
|MM	|номер месяца (01-12)|
|day	|полное название дня недели в нижнем регистре, дополненное пробелами до девяти символов|
|DDD	|номер дня в году (001-366)|
|DD	|день месяца (01-31)|
|ID	|номер дня недели по ISO 8601, считая с понедельника (1) до воскресенья (7)|
|WW	|номер недели в году (1-53); первая неделя начинается в первый день года|

Округление даты

In [67]:
run_query("SELECT now(), date_trunc('hour',now())")

,now,date_trunc
0,2023-01-19 09:15:11.094903+00:00,2023-01-19 09:00:00+00:00


* microseconds;
* milliseconds;
* second;
* minute;
* hour;
* day;
* week;
* month;
* quarter;
* year;
* decade;
* century;
* millennium.

In [70]:
run_query("SELECT '2019-01-01'::date + 500 as later_date")
run_query("SELECT '2019-02-10'::date - '2017-03-01'::date as diff_days")

,later_date
0,2020-05-15


,diff_days
0,711


Строка в дату

In [85]:
run_query("SELECT to_date('05 Dec 2000', 'DD Mon YYYY') as date")
run_query("SELECT to_timestamp('05 Dec 2000', 'DD Mon YYYY') as time")

,date
0,2000-12-05


,time
0,2000-12-05 00:00:00+00:00


#### Строки


* CHARACTER - строка фиксированной длины, дополненная пробелами;
* CHARACTER VARYING - строка ограниченной переменной длины;
* TEXT - строка неограниченной длины

Конкатенация

In [72]:
run_query("SELECT 'Всем ' || 'привет' as text") # -> тип text

,text
0,Всем привет


In [75]:
run_query("""
    select 'select * from '||t.table_schema||'.'||t.table_name||';' query
    from information_schema.tables t
    where table_schema = 'shipping'
""")

,query
0,select * from shipping.truck;
1,select * from shipping.city;
2,select * from shipping.customer;
3,select * from shipping.driver;
4,select * from shipping.shipment;


Изменение регистра

In [76]:
run_query("select upper('Abc') s1 ,lower('xYz') s2")

,s1,s2
0,ABC,xyz


Замена символов

In [77]:
run_query("select replace('Hello, world!',', world','')")

,replace
0,Hello!


Подстрока

In [78]:
run_query("""
with t as
(
select 'Один два три'::text sample_string
)
select 
 left(t.sample_string,4) one,
 right(left(t.sample_string,8),3) two,
 right(t.sample_string,3) three
from t
""")

,one,two,three
0,Один,два,три


In [79]:
run_query("select left('0123456789', - 2), right('0123456789', - 2)")

,left,right
0,01234567,23456789


Интерполяция

In [84]:
run_query("""
    SELECT format('driver_id = %s, first_name = %s, last_name = %s', driver_id, first_name, last_name) 
    FROM shipping.driver d 
    LIMIT 2
""")

,format
0,"driver_id = 30, first_name = Maria, last_name ..."
1,"driver_id = 29, first_name = Kristie, last_nam..."


In [86]:
connection.close()